In [1]:
# Initial imports
import numpy as np

import torch

from captum.attr import IntegratedGradients
from captum.attr import LayerConductance
from captum.attr import NeuronConductance

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

from scipy import stats
import pandas as pd

device = torch.device("cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu")

/Users/kpt/.pyenv/versions/kedro19/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Update path to dataset here.
dataset_path = "aligned_df.pq"
data_set = pd.read_parquet(dataset_path)

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import TensorDataset, DataLoader

# Hyper-parameters 

num_classes = 1
num_epochs = 5
batch_size = 32
learning_rate = 0.0001


# input_size = 5
sequence_length = 100 # the window it trains with can be selected
hidden_size = 32
# hidden_size = 256 
num_layers = 2
step_size = 1

# features = ['timestamp', 'A1_Sensor', 'A1_Sensor_diff', 'A1_Resistance', 'A1_Resistance_diff', 'A1_Sensor_norm', 'A1_Resistance_norm']
features = ['timestamp_bin', 'A1_Resistance', 'A1_Resistance_diff', 'A1_Resistance_norm']
target_column = 'resistance_ratio'

input_size = len(features)  # Number of features

# create sequences for each experiment
# sequence_length = 50  # Example sequence length

# features = ['timestamp', 'A1_Sensor', 'A1_Sensor_diff', 'A1_Resistance', 'A1_Resistance_diff', 'A1_Sensor_norm', 'A1_Resistance_norm']
features = ['timestamp_bin', 'A1_Resistance', 'A1_Resistance_diff', 'A1_Resistance_norm']
target_column = 'resistance_ratio'

input_size = len(features)  # Number of features

# Initialize lists to hold sequences and targets
sequences = []
targets = []
padding_length = 50  # Number of timesteps to pad at the end

# --------padding with LOCF---------
for _, group in data_set.groupby('exp_no'):
    # Only select the rows with the relevant columns
    data = group[features].values
    target_data = group[target_column].values
    
    # If there is data to pad
    if len(data) > 0:
        # Pad the end of the dataset with the last value for features
        pad_feature = np.repeat(data[-1, :][np.newaxis, :], padding_length, axis=0)
        data = np.vstack((data, pad_feature))
        
        # Pad the end of the dataset with the last value for targets
        pad_target = np.repeat(target_data[-1], padding_length)
        target_data = np.concatenate((target_data, pad_target))

    # Create sequences
    for i in range(len(data) - sequence_length):
        # Extract the sequence of features and the corresponding target
        sequence = data[i:(i + sequence_length)]
        target = target_data[i + sequence_length - 1]  # Target aligned with the end of the sequence
        
        sequences.append(sequence)
        targets.append(target)

# --------padding with LOCF---------

# ---------padding----------

# # Group by 'exp_no' and create sequences for each group
# for _, group in data_set.groupby('exp_no'):
#     # Only select the rows with the relevant columns
#     data = group[features].values
#     target_data = group[target_column].values
    
#     # Pad the end of the dataset with zeros for features
#     pad_feature = np.zeros((padding_length, len(features)))
#     data = np.vstack((data, pad_feature))
    
#     # Optionally, pad the end of the dataset with zeros or a specific value for targets
#     pad_target = np.zeros(padding_length)
#     target_data = np.concatenate((target_data, pad_target))

#     # Create sequences
#     for i in range(len(data) - sequence_length):
#         # Extract the sequence of features and the corresponding target
#         sequence = data[i:(i + sequence_length)]
#         target = target_data[i + sequence_length - 1]  # Target aligned with the end of the sequence
        
#         sequences.append(sequence)
#         targets.append(target)


# ---------padding----------




# ---------no padding----------


# # Group by 'exp_no' and create sequences for each group
# for _, group in data_set.groupby('exp_no'):
#     # Only select the rows with the relevant columns
#     data = group[features].values
#     target_data = group[target_column].values
    
#     # Create sequences
#     for i in range(len(group) - sequence_length):
#         # Extract the sequence of features and the corresponding target
#         sequence = data[i:(i + sequence_length)]
#         target = target_data[i + sequence_length]  # Target is the next record
        
#         sequences.append(sequence)
#         targets.append(target)


# ---------no padding----------

# Convert lists to numpy arrays
sequences_np = np.array(sequences)
targets_np = np.array(targets)

sequences_train, sequences_test, targets_train, targets_test = train_test_split(
    sequences_np, targets_np, test_size=0.2, random_state=42)


# Initialize the scaler
scaler = StandardScaler()

# Reshape, fit, and transform the training data
n_samples_train, sequence_length, n_features = sequences_train.shape
sequences_train_reshaped = sequences_train.reshape(-1, n_features)
scaler.fit(sequences_train_reshaped)  # Fit only on training data
sequences_train_scaled = scaler.transform(sequences_train_reshaped).reshape(n_samples_train, sequence_length, n_features)

# Transform the testing data
n_samples_test, _, _ = sequences_test.shape
sequences_test_reshaped = sequences_test.reshape(-1, n_features)
sequences_test_scaled = scaler.transform(sequences_test_reshaped).reshape(n_samples_test, sequence_length, n_features)

# Convert numpy arrays to float32 before converting to PyTorch tensors
sequences_np = sequences_np.astype(np.float32)
targets_np = targets_np.astype(np.float32)

train_sequences_tensor = torch.tensor(sequences_train_scaled, dtype=torch.float32)
test_sequences_tensor = torch.tensor(sequences_test_scaled, dtype=torch.float32)

train_targets_tensor = torch.tensor(targets_train, dtype=torch.float32)
test_targets_tensor = torch.tensor(targets_test, dtype=torch.float32)

# Create TensorDatasets
train_dataset = TensorDataset(train_sequences_tensor, train_targets_tensor)
test_dataset = TensorDataset(test_sequences_tensor, test_targets_tensor)

# Create DataLoaders
# batch_size = batch_size  # You can adjust this based on your memory constraints
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)



In [4]:
import torch
import torch.nn as nn

# Fully connected neural network with one hidden layer
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNN, self).__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)
        
    def forward(self, x):
        # Set initial hidden states (and cell states for LSTM)
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device) 
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device) 
        out, _ = self.lstm(x, (h0,c0))  
        out = out[:, -1, :]
        out = self.fc(out)
        return out

In [5]:
model = RNN(input_size, hidden_size, num_layers, num_classes).to(device)
checkpoint = torch.load('checkpoints/checkpoint_epoch_5_20240410-102415.pt', map_location=torch.device('cpu'))
model_state_dict = checkpoint['model_state_dict']
model.load_state_dict(model_state_dict)

<All keys matched successfully>

In [6]:
import math

model.eval()  # Ensure the model is in evaluation mode
model.to(device)  # Move the model to the device

criterion = nn.MSELoss()  # For regression tasks
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

training_losses = []
validation_rmses = []

total_val_loss = 0
count = 0
with torch.no_grad():
    for sequences_batch, targets_batch in test_loader:
        sequences_batch = sequences_batch.to(device)
        targets_batch = targets_batch.to(device).unsqueeze(-1)
            
        outputs = model(sequences_batch)
        loss = criterion(outputs, targets_batch)
            
        total_val_loss += loss.item()
        count += 1
    
avg_val_loss = total_val_loss / count
val_rmse = math.sqrt(avg_val_loss)
validation_rmses.append(val_rmse)
    
print(f'Test RMSE: {val_rmse}')


Validation RMSE: 0.027634603832432592


In [7]:
ig = IntegratedGradients(model)

In [10]:
test_sequences_tensor = test_sequences_tensor.to(device)
test_sequences_tensor.requires_grad_()
attr, delta = ig.attribute(test_sequences_tensor,target=1, return_convergence_delta=True)
attr = attr.detach().numpy()

RuntimeError: MPS backend out of memory (MPS allocated: 8.92 GB, other allocations: 32.72 MB, max allowed: 9.07 GB). Tried to allocate 139.27 MB on private pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).